In [1]:
import os
import joblib
import tqdm
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, StratifiedKFold

In [2]:
from mmfe.MMFE import MMFE, normalize, denormalize, get_metrics

In [3]:
from sklearn.linear_model import ElasticNet, Ridge, Lasso
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
import xgboost as xgb

In [4]:
random_state = 2025
np.random.seed(random_state)
torch.manual_seed(random_state)
if torch.cuda.is_available():
    torch.cuda.manual_seed(random_state)
    torch.cuda.manual_seed_all(random_state)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [5]:
sns.set_theme(style='whitegrid')

# 1. Initialization

## Task

In [6]:
TASKNAME = '300nM'

output_dir = os.path.join('results', TASKNAME)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

output_dir = os.path.join(output_dir, 'MMFE')
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

## Torch Device

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Data Load

In [8]:
df_raw = pd.read_csv(os.path.join('data', 'selectivity.csv'))

In [9]:
df_raw

,Compound,Drug name,PubChem CID,Binding Mode (based on ABL1-phos. vs. -nonphos affinity),S(300nM),S(3uM),SMILES
0,A-674563,NaN,11314340,undetermined,0.1166,0.2772,CC1=C2C=C(C=CC2=NN1)C3=CC(=CN=C3)OC[C@H](CC4=C...
1,AB-1010,Masitinib,10074640,Type II,0.0337,0.0622,CC1=C(C=C(C=C1)NC(=O)C2=CC=C(C=C2)CN3CCN(CC3)C...
2,ABT-869,Linifanib,11485656,undetermined,0.0648,0.1839,CC1=CC(=C(C=C1)F)NC(=O)NC2=CC=C(C=C2)C3=C4C(=C...
3,AC220,Quizartinib,24889392,Type II,0.0285,0.0751,CC(C)(C)C1=CC(=NO1)NC(=O)NC2=CC=C(C=C2)C3=CN4C...
4,AG-013736,Axitinib,6450551,Type I,0.0570,0.1969,CNC(=O)C1=CC=CC=C1SC2=CC3=C(C=C2)C(=NN3)/C=C/C...
...,...,...,...,...,...,...,...
67,TG-100-115,NaN,10427712,Type I,0.0337,0.1321,C1=CC(=CC(=C1)O)C2=NC3=C(N=C(N=C3N=C2C4=CC(=CC...
68,TG-101348,NaN,16722836,Type I,0.1788,0.5389,CC1=CN=C(N=C1NC2=CC(=CC=C2)S(=O)(=O)NC(C)(C)C)...
69,Vandetanib,NaN,3081361,Type I,0.0933,0.2358,CN1CCC(CC1)COC2=C(C=C3C(=C2)N=CN=C3NC4=C(C=C(C...
70,VX-680/MK-0457,Tozasertib,5494449,Type I,0.1321,0.3472,CC1=CC(=NN1)NC2=CC(=NC(=N2)SC3=CC=C(C=C3)NC(=O...


In [10]:
X_raw = df_raw['SMILES'].values
y_raw = df_raw[f'S({TASKNAME})'].values.astype(np.float32)

# 2. 5-Fold Cross-Validation

In [11]:
# 5-fold cross-validation setup
y_binned = pd.qcut(y_raw, q=5, labels=False, duplicates='drop')
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state)

ys = []

for fold, (train_idx, test_idx) in enumerate(tqdm.tqdm(skf.split(X_raw, y_binned), total=5, desc="5-Fold CV")):
    ###################################################################
    ## Output Directory
    ###################################################################
    # Create fold directory
    fold_dir = os.path.join(output_dir, f'Fold{fold+1}')
    if not os.path.exists(fold_dir):
        os.makedirs(fold_dir)
        
    ###################################################################
    ## Data Splits
    ###################################################################
    # Test set
    X_te = X_raw[test_idx]
    y_te = y_raw[test_idx]
    
    # Train set (further split into train and validation)
    X_train_full = X_raw[train_idx]
    y_train_full = y_raw[train_idx]
    
    # Split train into train and validation
    idx_tr, idx_va = train_test_split(range(len(X_train_full)), test_size=0.1, random_state=2025)
    
    X_tr = X_train_full[idx_tr]
    y_tr = y_train_full[idx_tr]
    X_va = X_train_full[idx_va]
    y_va = y_train_full[idx_va]

    ###################################################################
    ## Model Initialization
    ###################################################################
    model = MMFE(output_dir, device, fold)

    ###################################################################
    ## Model Training
    ###################################################################
    _ = model.fit(X_tr, X_va, y_tr.reshape(-1,1), y_va.reshape(-1,1), temperature=1.0)

    ###################################################################
    ## Embeddings
    ###################################################################
    H_tv = model.predict(np.hstack([X_tr, X_va]))
    z_tv = normalize(np.hstack([y_tr, y_va]))
    H_te = model.predict(X_te)

    ###################################################################
    ## Classifier
    ###################################################################
    clfs = {
        'ElasticNet': ElasticNet(alpha=0.01, random_state=random_state),
        'Ridge': Ridge(random_state=random_state),
        'Lasso': Lasso(alpha=0.01, random_state=random_state),
        'SVR': SVR(),
        'KNN': KNeighborsRegressor(),
        'DecisionTree': DecisionTreeRegressor(random_state=random_state),
        'RandomForest': RandomForestRegressor(random_state=random_state),
        'AdaBoost': AdaBoostRegressor(random_state=random_state),
        'XGBoost': xgb.XGBRegressor(random_state=random_state)
    }

    # Process each test sample for this fold
    fold_results = []
    for i, (gt, h_te) in enumerate(zip(y_te, H_te)):
        p_te = {'GroundTruth': gt, 'Fold': fold}
        for clf_name, clf in clfs.items():
            clf.fit(H_tv, z_tv)
            p_te[clf_name] = denormalize(clf.predict(h_te.reshape(1, -1))[0])
            ## joblib
            with open(os.path.join(fold_dir, f"{clf_name}.pkl"), "wb") as f:
                joblib.dump(clf, f, protocol=5)
        fold_results.append(p_te)
        ys.append(p_te)
    
    ###################################################################
    ## Save individual fold results
    ###################################################################
    # Save fold predictions
    df_fold = pd.DataFrame(fold_results)
    df_fold.to_csv(os.path.join(fold_dir, 'predictions.csv'), index=False)
    
    # Calculate and save fold metrics
    fold_metrics = []
    model_names = [col for col in df_fold.columns if col not in ['GroundTruth', 'Fold']]
    
    for model_name in model_names:
        fold_res = get_metrics(df_fold['GroundTruth'], df_fold[model_name])
        fold_res['model_name'] = model_name
        fold_metrics.append(fold_res)
    
    df_fold_metrics = pd.DataFrame(fold_metrics)
    df_fold_metrics.to_csv(os.path.join(fold_dir, 'metrics.csv'), index=False)

5-Fold CV:   0%|                                                                                                                                                         | 0/5 [00:00<?, ?it/s]/home/descartes/miniconda3/envs/mmfe/lib/python3.9/site-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1659484806139/work/aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
5-Fold CV: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [01:59<00:00, 23.86s/it]


In [12]:
df_ys = pd.DataFrame(ys)

In [13]:
df_ys

,GroundTruth,Fold,ElasticNet,Ridge,Lasso,SVR,KNN,DecisionTree,RandomForest,AdaBoost,XGBoost
0,0.0337,0,0.026816,0.026580,0.027506,0.025368,0.024352,0.0233,0.027117,0.032827,0.024522
1,0.2617,0,0.101648,0.101760,0.099137,0.112838,0.075964,0.1788,0.107158,0.117935,0.099665
2,0.0803,0,0.043931,0.046862,0.046100,0.046275,0.050504,0.0544,0.047318,0.047827,0.054030
3,0.0518,0,0.101133,0.102765,0.102657,0.117282,0.130906,0.1632,0.131264,0.117935,0.124561
4,0.0285,0,0.040182,0.041250,0.040042,0.038373,0.044371,0.0233,0.043445,0.038845,0.040641
...,...,...,...,...,...,...,...,...,...,...,...
67,0.0544,4,0.113535,0.112281,0.113209,0.121513,0.137650,0.0596,0.100943,0.130100,0.165841
68,0.0311,4,0.059921,0.063050,0.054400,0.073737,0.070578,0.0596,0.071259,0.056502,0.070355
69,0.3420,4,0.100258,0.094592,0.096923,0.098823,0.083137,0.2824,0.072474,0.079148,0.141024
70,0.3109,4,0.224982,0.227673,0.224050,0.230099,0.174825,0.1632,0.220001,0.221475,0.249257


In [14]:
results = []
model_names = [col for col in df_ys.columns if col not in ['GroundTruth', 'Fold']]

for model_name in model_names:
    fold_metrics = []
    for fold in range(5):
        fold_data = df_ys[df_ys['Fold'] == fold]
        if len(fold_data) > 0:
            fold_res = get_metrics(fold_data['GroundTruth'], fold_data[model_name])
            fold_metrics.append(fold_res)
    
    # Calculate mean and std across folds
    if fold_metrics:
        rmse_values = [m['rmse'] for m in fold_metrics]
        r2_values = [m['r2'] for m in fold_metrics]
        pcc_values = [m['pcc'] for m in fold_metrics]
        
        res = {
            'model_name': model_name,
            'rmse': round(np.mean(rmse_values), 3),
            'rmse_std': round(np.std(rmse_values), 3),
            'r2': round(np.mean(r2_values), 3),
            'r2_std': round(np.std(r2_values), 3),
            'pcc': round(np.mean(pcc_values), 3),
            'pcc_std': round(np.std(pcc_values), 3)
        }
        results.append(res)

df_res = pd.DataFrame(results)

In [15]:
df_res.to_csv(os.path.join(output_dir, 'metrics.csv'), index=False)

In [16]:
df_res

,model_name,rmse,rmse_std,r2,r2_std,pcc,pcc_std
0,ElasticNet,0.110,0.021,-0.040,0.490,0.386,0.406
1,Ridge,0.111,0.021,-0.049,0.499,0.384,0.406
2,Lasso,0.111,0.020,-0.045,0.483,0.376,0.410
3,SVR,0.111,0.020,-0.016,0.408,0.364,0.388
4,KNN,0.115,0.019,-0.132,0.556,0.329,0.438
5,DecisionTree,0.131,0.011,-0.741,1.121,0.278,0.417
6,RandomForest,0.116,0.016,-0.188,0.629,0.360,0.360
7,AdaBoost,0.125,0.015,-0.443,0.928,0.347,0.341
8,XGBoost,0.124,0.033,-0.619,1.379,0.413,0.380
